In [1]:
from pathlib import Path

from tqdm import tqdm

import pandas as pd
import numpy as np

from sklearn.model_selection import RepeatedKFold
from pynndescent import NNDescent

## Prepare path for the results

In [2]:
results_path_pattern = 'results_ArdiMiPE/{}.csv'
# results_path_pattern = 'results_3mers/{}.csv'

In [3]:
Path(results_path_pattern).parent.mkdir(exist_ok=True)

## Load data

In [4]:
swiss_df = pd.read_hdf('../data/bacterial_swissprot.h5')
swiss_df.head()

,accessions,sequence_length,sequence,description,InterPro,GO,KO,Gene3D,Pfam,KEGG,...,EC number,Superkingdom,Kingdom,Phylum,Class,Order,Family,Subfamily,Genus,Species
entry_name,,,,,,,,,,,,,,,,,,,,,
12AH_CLOS4,P21215,29.0,MIFDGKVAIITGGGKAKSIGYGIAVAYAK,RecName: Full=12-alpha-hydroxysteroid dehydrog...,IPR036291,GO:0047013||GO:0030573||GO:0016042,NaN,NaN,NaN,NaN,...,1.1.1.176,Bacteria,NaN,Firmicutes,Clostridia,Clostridiales,Clostridiaceae,NaN,Clostridium,NaN
12KD_MYCSM,P80438,24.0,MFHVLTLTYLCPLDVVXQTRPAHV,RecName: Full=12 kDa protein; Flags: Fragment;,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Bacteria,NaN,Actinobacteria,Actinobacteria,Corynebacteriales,Mycobacteriaceae,NaN,Mycolicibacterium,NaN
12OLP_LISIN,Q92AT0,1086.0,MTMLKEIKKADLSAAFYPSGELAWLKLKDIMLNQVIQNPLENRLSQ...,"RecName: Full=1,2-beta-oligoglucan phosphoryla...",IPR008928||IPR012341||IPR033432,GO:0016740,K21298,1.50.10.10,PF17167,lin:lin1839,...,2.4.1.333,Bacteria,NaN,Firmicutes,Bacilli,Bacillales,Listeriaceae,NaN,Listeria,NaN
12S_PROFR,Q8GBW6||Q05617,611.0,MAENNNLKLASTMEGRVEQLAEQRQVIEAGGGERRVEKQHSQGKQT...,RecName: Full=Methylmalonyl-CoA carboxyltransf...,IPR034733||IPR000438||IPR029045||IPR011763||IP...,GO:0009317||GO:0003989||GO:0047154||GO:0006633,NaN,NaN,PF01039,NaN,...,2.1.3.1,Bacteria,NaN,Actinobacteria,Actinobacteria,Propionibacteriales,Propionibacteriaceae,NaN,Propionibacterium,NaN
14KD_MYCBO,P0A5B8||A0A1R3Y251||P30223||X2BJK6,144.0,MATTLPVQRHPRSLFPEFSELFAAFPSFAGLRPTFDTRLMRLEDEM...,RecName: Full=14 kDa antigen; AltName: Full=16...,IPR002068||IPR008978,GO:0005618||GO:0005576,NaN,2.60.40.790,PF00011,NaN,...,NaN,Bacteria,NaN,Actinobacteria,Actinobacteria,Corynebacteriales,Mycobacteriaceae,NaN,Mycobacterium,NaN


In [5]:
embed_df = pd.read_hdf('../data/reduced_deep_embeddings.h5')
# embed_df = pd.read_hdf('../data/reduced_3mers_embeddings.h5')
embedding_columns = embed_df.columns

embed_df.head()

,pca_0,pca_1,pca_2,pca_3,pca_4,pca_5,pca_6,pca_7,pca_8,pca_9,...,pca_40,pca_41,pca_42,pca_43,pca_44,pca_45,pca_46,pca_47,pca_48,pca_49
entry_name,,,,,,,,,,,,,,,,,,,,,
12AH_CLOS4,-1.423302,-0.736753,-0.874685,-0.819151,-0.142118,-0.816148,0.386158,0.061937,0.687497,-0.297038,...,0.118606,-0.178684,0.080721,0.000936,-0.040333,0.036337,-0.045772,0.040853,-0.182722,0.036003
12KD_MYCSM,-2.985445,0.512436,-0.804671,-0.881089,0.023167,-0.840529,0.681208,-0.202686,0.343030,0.302511,...,0.063164,0.068167,-0.055077,0.054813,0.115826,-0.138364,-0.137902,-0.151101,-0.352669,-0.107964
12OLP_LISIN,-3.489646,0.009758,0.592749,-0.249326,-0.399583,0.004498,-0.739661,0.568125,-0.138422,-0.558067,...,0.078285,-0.139291,0.163776,-0.086977,0.097379,-0.009465,-0.043559,-0.039196,-0.102595,-0.001293
12S_PROFR,0.622390,-0.471249,0.868454,0.555548,-0.403828,-0.031662,-0.045285,-0.540816,0.314709,-0.140730,...,-0.212403,0.026798,-0.185727,-0.201333,-0.117253,-0.084229,-0.140449,-0.099275,0.174085,0.006138
14KD_MYCBO,-0.739342,0.083417,-0.164653,-0.817445,-0.376513,-0.484122,-0.727681,0.142234,0.060500,0.264555,...,-0.006761,-0.284457,0.022018,0.025971,0.017194,-0.175638,0.002390,-0.232771,-0.178814,0.032900


In [6]:
swiss_df = swiss_df.join(embed_df)

## Helpers

In [7]:
def calculate_iou(ground_truth, predictions):
    predictions_counts = pd.Series(predictions).value_counts()
    len_ground_truth = len(ground_truth)
    
    # take all values that are equally popular as the last value - we break ties in this manner
    most_popular_values = predictions_counts.loc[
        predictions_counts >= predictions_counts.iloc[
            min(len_ground_truth-1, len(predictions_counts)-1)
        ]
    ].index
    
    intersection = set(most_popular_values).intersection(set(ground_truth))
    union = set(most_popular_values).union(set(ground_truth))
    return len(intersection) / len(union)

def calculate_metric(neighbors, train_y, test_y, metric_function):
    neighbor_labels = train_y.values[neighbors]

    labels_df = (
        pd.DataFrame(neighbor_labels, index=test_y.index)
            .sum(axis=1)
            .rename('neighbors_labels')
            .to_frame()
    )
    
    labels_df = labels_df.merge(test_y, left_index=True, right_index=True)
    
    return labels_df.apply(lambda x: metric_function(x[1], x[0]), axis=1)

## Cross-validation for all annotations 

In [8]:
ks = [3, 15, 51]
n_repeats = 10
n_folds = 5
random_state=0

annotations = [
    'Phylum',
    'Order',
    'Family',
    'Genus',
    'SUPFAM',
    'Gene3D',
    'InterPro',
    'KO',
    'GO',
    'eggNOG',
    'Pfam',
    'EC number'
]
swiss_df[annotations].head()

,Phylum,Order,Family,Genus,SUPFAM,Gene3D,InterPro,KO,GO,eggNOG,Pfam,EC number
entry_name,,,,,,,,,,,,
12AH_CLOS4,Firmicutes,Clostridiales,Clostridiaceae,Clostridium,SSF51735,NaN,IPR036291,NaN,GO:0047013||GO:0030573||GO:0016042,NaN,NaN,1.1.1.176
12KD_MYCSM,Actinobacteria,Corynebacteriales,Mycobacteriaceae,Mycolicibacterium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12OLP_LISIN,Firmicutes,Bacillales,Listeriaceae,Listeria,SSF48208,1.50.10.10,IPR008928||IPR012341||IPR033432,K21298,GO:0016740,ENOG4107T40||COG3459,PF17167,2.4.1.333
12S_PROFR,Actinobacteria,Propionibacteriales,Propionibacteriaceae,Propionibacterium,SSF52096,NaN,IPR034733||IPR000438||IPR029045||IPR011763||IP...,NaN,GO:0009317||GO:0003989||GO:0047154||GO:0006633,ENOG4107QX3||COG4799,PF01039,2.1.3.1
14KD_MYCBO,Actinobacteria,Corynebacteriales,Mycobacteriaceae,Mycobacterium,SSF49764,2.60.40.790,IPR002068||IPR008978,NaN,GO:0005618||GO:0005576,NaN,PF00011,NaN


In [ ]:
for annotation in tqdm(annotations): 
    print(annotation)

    # Not null annotation column and embedding columns
    annot_df = swiss_df[
        swiss_df[annotation].notnull() & swiss_df[embedding_columns].notnull().all(axis=1)
    ]#.sample(1000) # For testing
   
    metrics_df = pd.DataFrame(
        data=np.nan,
        index=annot_df.index,
        columns=[f'k={k}' for k in ks]
    )
    
    kfold = RepeatedKFold(n_splits=n_folds, random_state=random_state, n_repeats=n_repeats)
    for train_ids, test_ids in tqdm(kfold.split(annot_df), total=kfold.get_n_splits()):
        
        # Train-test split
        train_X = annot_df.iloc[train_ids][embedding_columns]
        test_X = annot_df.iloc[test_ids][embedding_columns]
        train_y = annot_df.iloc[train_ids][annotation]
        test_y = annot_df.iloc[test_ids][annotation]
        
        # Expand labels to lists
        train_y, test_y = train_y.str.split(pat=r'\|\|'), test_y.str.split(pat=r'\|\|')

        # Build & query NN graph
        nn_graph = NNDescent(train_X, n_neighbors=max(ks), n_jobs=8)
        
        neighbors, distances = nn_graph.query(test_X, k=max(ks))

        for k in ks:
            k_nearest_neighbors = neighbors[:, :k]
            metric_values = calculate_metric(k_nearest_neighbors, train_y, test_y, calculate_iou)
            
            metrics_df.loc[metric_values.index, f'k={k}'] = metric_values.values            
            print(f'{annotation} for k={k}: {metric_values.mean():.4f}')

        metrics_df.to_csv(results_path_pattern.format(annotation))

  0%|          | 0/50 [00:00<?, ?it/s]

Phylum
Phylum for k=3: 0.8215
Phylum for k=15: 0.7574
Phylum for k=51: 0.6823



  2%|▏         | 1/50 [02:40<2:10:55, 160.31s/it]

Phylum for k=3: 0.8210
Phylum for k=15: 0.7562
Phylum for k=51: 0.6807



  4%|▍         | 2/50 [05:05<2:04:42, 155.88s/it]

...

Order for k=3: 0.5642
Order for k=15: 0.4467
Order for k=51: 0.3142



 10%|█         | 5/50 [11:38<1:45:00, 140.02s/it]

Order for k=3: 0.5611
Order for k=15: 0.4429
Order for k=51: 0.3129



 12%|█▏        | 6/50 [13:57<1:42:29, 139.77s/it]

Order for k=3: 0.5690
